<a id = 'intro'></a>
# Introduction

League of Legends is an online-multiplayer video game consisting of two, five player teams. The objective of the game is to destroy the enemy team's nexus. To gain an advantage, teams collect gold through kills and objectives. The game is highly competitive and requires teamwork.

In this analysis, we will look at data consisting of the first 10 minutes of Diamond ranked games. We will use Sklearn to cluster the data using all the available clustering algorithms. The purpose of this project is to familiarize myself with the Sklean clustering API and to show others how simple it is to use it.

<a id = 'top'></a>
# Table of Contents

* [Introduction](#intro)

* [Data Cleaning](#cleaning)
    * [Remove Repeated Columns](#repeated_columns)
    * [Remove Outliers](#remove_outliers)
    * [Rescaling Features](#rescaling)
    * [Remove Null Values](#null)
   
* [Clustering](#clusters)
    * [K-Means](#kmeans)
    * [Affinity Propagation](#affinity)
    * [Mean Shift](#mean_shift)
    * [Spectral Clustering](#spectral)
    * [Agglomerative Clustering](#agglomerative)
        * [Ward](#ward)
        * [Complete](#complete)
        * [Average](#average)
        * [Single](#single)
    * [DSCAN](#dbscan)
    * [OPTICS](#optics)
    * [Birch](#birch)

* [Clustering Discussion](#discussion)
* [Team Advantage Column](#advantage)

<a id = 'cleaning'></a>
# Data Cleaning

We begin our analysis with reading the csv file and inspecting its columns.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
sns.set_style('darkgrid')

RED = '#D10000'
BLUE = '#0082FF'

In [ ]:
data = pd.read_csv('../input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv')
data.head()

In [ ]:
# get the column list
data.info()

We can see that this dataset consists of 40 columns. Some of the columns are named differently, but contain the same information. For example, the "blueKills" column should be the same as the "redDeaths" column. Afterall, if a blue team gets a kill, then a red team member must have been killed.

<a id = 'repeated_columns'></a>
## Remove Repeated Columns

Next we can see a list of potential repeated columns:
* blueFirstBlood is the opposite of redFirstBlood. If blue team gets first blood, the column is assigned a 1 and 0 for the red team. The opposite is true for red team.
* blueKills is redDeaths
* blueDeaths is redKills
* blueExperienceDiff is the negative of redExperienceDiff
* blueGoldDiff is the negative redGoldDiff

Below we verify that these relationships are true.

In [ ]:
# first blood
all(data['blueFirstBlood'] == data['redFirstBlood'].apply(lambda x: 0 if x == 1 else 1))

In [ ]:
# blue kills is red deaths
all(data['blueKills'] == data['redDeaths'])

In [ ]:
# blue deaths is red kills
all(data['blueDeaths'] == data['redKills'])

In [ ]:
# blue experience difference is the negative of red experience difference
all(data['blueExperienceDiff'] == data['redExperienceDiff'].apply(lambda x: -1*x))

In [ ]:
# blue gold difference is the negative of red gold difference
all(data['blueGoldDiff'] == data['redGoldDiff'].apply(lambda x: -1*x))

The decrease the amount of features, we will drop the repeated columns that contain red team information. We are keeping the blue team columns just because of the "blueWins" feature. We will also drop the "gameId", since it does not contribute to the characteristics of the games.

In [ ]:
data.drop(['redFirstBlood','redDeaths','redKills','redExperienceDiff','redGoldDiff', 'gameId'],
          axis = 1, inplace = True)

Now we gather the blue and red team columns to get a rough idea of their distributions.

In [ ]:
blue_team = [column for column in data.columns if 'blue' in column]
red_team = [column for column in data.columns if 'red' in column]

In [ ]:
data[blue_team].hist(color = BLUE,
                     figsize = (10,10))
plt.tight_layout()
plt.show()

From the histogram above, we can see that the columns that are approximately normal are blueKills, blueDeaths, blueAssists, blueTotalGold, blueAvgLevel, blueTotalExperience, blueTotalMinionsKilled, blueTotalJungleMinionsKilled, blueGoldDiff, blueExperienceDiff, blueCSPerMin and blueGoldPerMin.

In [ ]:
data[red_team].hist(color = RED,
                    figsize = (10,10))

plt.tight_layout()
plt.show()

From the histogram above, we can see similar normal distributions in the red team columns: redAssists, redTotalGold, redAvgLevel, redTotalExperience, redTotalMinionsKilled, redTotalJungleMinionsKilled, redCSPerMin, and redGoldPerMin.

[back to top](#top)

<a id = 'remove_outliers'></a>
## Remove Outliers

Next, we will visualize the features using boxplots and remove any statistical outliers. For this analysis, we will keep any data points that are within 3 standard deviations from the mean.

We start with the blue team.

In [ ]:
# collect the columns that are "normally distributed".
normal_columns = ['blueWardsPlaced','blueWardsDestroyed','blueKills','blueDeaths',
                  'blueAssists','blueTotalGold','blueAvgLevel','blueTotalExperience',
                  'blueTotalMinionsKilled', 'blueTotalJungleMinionsKilled',
                  'blueGoldDiff','blueExperienceDiff','blueCSPerMin','blueGoldPerMin',
                  'redWardsPlaced','redAssists','redTotalGold','redAvgLevel','redTotalExperience',
                  'redTotalMinionsKilled','redTotalJungleMinionsKilled',
                  'redCSPerMin','redGoldPerMin']

# we need to group the columns with similar range so that the boxplots look interpretable
blue_normal = [column for column in normal_columns if 'blue' in column]
data[blue_normal].describe().T

In [ ]:
# group the blue columns by relative size
small_blue = ['blueWardsPlaced','blueWardsDestroyed','blueKills',
              'blueDeaths','blueAssists','blueAvgLevel','blueTotalMinionsKilled',
              'blueTotalJungleMinionsKilled','blueCSPerMin']
large_blue = [column for column in blue_normal if column not in small_blue]

In [ ]:
def plot_boxplot(data, subset, color, title):
    plt.figure(figsize = (11,9))
    g = sns.boxplot(data = data[subset], color = color)
    g.set_title(title)
    plt.xticks(rotation = 30)
    plt.show()

In [ ]:
# now we visualize
plot_boxplot(data, small_blue, BLUE, 'Outlier Analysis for Blue Team with Small Values')

In [ ]:
plot_boxplot(data, large_blue, BLUE, 'Outlier Analysis for Blue Team with Large Values')

In both of the boxplots above, we can see many outliers. As mentioned before, we will remove data points that are 3 standard deviations away from the mean.

In [ ]:
def remove_outliers(data):
    """
    Remove the outlier if it is 3 standard deviations away from the mean
    """
    for column in data.columns:
        column_mean = data[column].mean()
        column_std = data[column].std()
        threshold = column_std*3 # three standard deviations
        lower, upper = column_mean - threshold, column_mean + threshold
        data = data[(data[column] >= lower) & (data[column] <= upper)]
    return data

In [ ]:
# remove the outliers for the blue team
data_no_outliers = remove_outliers(data)

Let's visualize the same features after the outliers have been removed.

In [ ]:
# now we visualize after removing outliers
plot_boxplot(data_no_outliers, small_blue, BLUE, 'Boxplot After Removing Statistical Outliers for Blue Team with Small Values')

In [ ]:
plot_boxplot(data_no_outliers, large_blue, BLUE, 'Boxplot After Removing Statistical Outliers for Blue Team with Large Values')

Let's do the same thing to the red team.

In [ ]:
red_normal = [column for column in normal_columns if 'red' in column]
data[red_normal].describe().T

In [ ]:
small_red = ['redWardsPlaced','redAssists','redAvgLevel',
             'redTotalMinionsKilled','redTotalJungleMinionsKilled', 
             'redCSPerMin']
large_red = [column for column in red_normal if column not in small_red]

In [ ]:
# visualize small red
plot_boxplot(data, small_red, RED, 'Outlier Analysis for Red Team with Small Values')

In [ ]:
plot_boxplot(data, large_red, RED, 'Outlier Analysis for Red Team with Large Values')

Now we remove the outliers.

In [ ]:
plot_boxplot(data_no_outliers, small_red, RED, 'Boxplot After Removing Statistical Outliers for Red Team with Small Values')

In [ ]:
plot_boxplot(data_no_outliers, large_red, RED, 'Boxplot After Removing Statistical Outliers for Red Team with Large Values')

[back to top](#top)

<a id = 'rescaling'></a>
## Rescaling Features

We will use Sklearn's StandardScaler to rescale the data.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_tr = pd.DataFrame(scaler.fit_transform(data_no_outliers.drop('blueWins', axis = 1)), 
                       columns = data_no_outliers.drop('blueWins', axis = 1).columns)

# add the bluewins column back in
data_tr['blueWins'] = data_no_outliers['blueWins']

In [ ]:
# verify that the columns are scaled
data_tr.describe().T

[back to top](#top)

<a id = 'null'></a>
## Remove Null-Values

We can see above that the blueWins column has 6330 non-null values compared to the other features which have 7888 non-null values. Some of the values were dropped. We will remove the columns that contain null values.

In [ ]:
data_tr.dropna(inplace = True, axis = 0)
data_tr.describe().T

[back to top](#top)

<a id = 'clusters'></a>
# Clustering

Sklearn's API allows us to build many unsupervised learning models easily. For this analysis, we will use all of the unsupervised learning algorithms that sklearn supports. This includes: K-Means, Affinity Propagation, Mean Shift, Spectral Clustering, Agglomerative Clustering, DBSCAN, OPTICS, BIRCH, and Gaussian Mixtures. A summary of how each clustering algorithm works is found in the documentation [here](https://scikit-learn.org/stable/modules/clustering.html).


I would like to see how long it takes to run these clustering algorithms. Below we define a function that will time the algortihms for us.

For each clustering algorithm, we will plot the Blue Team's total gold earned versus the Red Team's.

In [ ]:
def FormatIt(delta):
    """
    Input is a datetime object. We will format it based on how many
    seconds it contains
    """
    message = 'Time elapsed is '
    # format the seconds
    if (delta.seconds == 0) & (delta.microseconds < 1000000):
        print(message + '{} microseconds'.format(delta.microseconds))
  
    elif delta.seconds < 60:
        print(message + '{} seconds'.format(delta.seconds))
    
    elif delta.seconds < 3600:
        minutes = int(delta.seconds / 60)
        seconds = delta.seconds % 60
        print(message + '{} minutes {} seconds'.format(minutes, seconds))
      
    else:
        hours = int(delta.seconds / 3600)
        minutes = int(delta.seconds / 60)
        seconds = delta.seconds % 60
        print(message + '{} hours {} minutes {} seconds'.format(hours, minutes, seconds))

class TimeIt():
    """
    So we can find out how long the algorithms take to run
    """
    def __init__(self):
        """
        Get the current time
        """
        self.start = datetime.now()
    
    def stop(self):
        self.end = datetime.now()
        self.delta = self.end - self.start
        FormatIt(self.delta)
        
class Clusters():
    """
    So that we can plot the clusters of each algorithm easily
    """
    def __init__(self, cluster_name, cluster_list, data):
        self.cluster_name = cluster_name
        self.cluster_list = cluster_list
        self.cluster_labels = [model.labels_ for model in self.cluster_list]
        self.data = data.copy()
                
    def plot(self):
        fig, axs = plt.subplots(2,2, figsize = (11,9))
        
        # add the first cluster label to the df
        self.data.insert(0, 'Clusters', self.cluster_labels[0])
        
        # first scatter plot
        g = sns.scatterplot(data = self.data,
                            x = 'blueTotalGold',
                            y = 'redTotalGold',
                            hue = 'Clusters',
                            alpha = 0.5,
                            edgecolor = 'k',
                            palette = 'viridis',
                            ax = axs[0][0])
        
        # replace the cluster labels for the next model
        self.data['Clusters'] = self.cluster_labels[1]
        
        # second scatter plot
        g = sns.scatterplot(data = self.data,
                            x = 'blueTotalGold',
                            y = 'redTotalGold',
                            hue = 'Clusters',
                            alpha = 0.5,
                            edgecolor = 'k',
                            palette = 'viridis',
                            ax = axs[0][1])
        
        # replace the cluster labels for the next model
        self.data['Clusters'] = self.cluster_labels[2]
        
        # third scatter plot
        g = sns.scatterplot(data = self.data,
                            x = 'blueTotalGold',
                            y = 'redTotalGold',
                            hue = 'Clusters',
                            alpha = 0.5,
                            edgecolor = 'k',
                            palette = 'viridis',
                            ax = axs[1][0])
        
        # replace the cluster labels for the next model
        self.data['Clusters'] = self.cluster_labels[3]
        
        # fourth scatter plot
        g = sns.scatterplot(data = self.data,
                            x = 'blueTotalGold',
                            y = 'redTotalGold',
                            hue = 'Clusters',
                            alpha = 0.5,
                            edgecolor = 'k',
                            palette = 'viridis',
                            ax = axs[1][1])
        
        title = 'Scatterplots for {}'.format(self.cluster_name)
        fig.suptitle(title, fontsize = 16)
        fig.tight_layout()
        fig.subplots_adjust(top=0.88)
        plt.show()

<a id = 'kmeans'></a>
## K-Means Clustering

According to the Sklearn documentation, "The KMeans algorithm clusters data by trying to separate samples in n groups of equal variance, minimizing a criterion known as the inertia or within-cluster sum-of-squares (see below). This algorithm requires the number of clusters to be specified. It scales well to large number of samples and has been used across a large range of application areas in many different fields."

We will run 4 different K-Means Clustering algorithms using 2,3,4, and 5 clusters.

In [ ]:
# make a copy so we don't change the original df
data_copy = data_tr.copy()
data_not_scaled = pd.DataFrame(scaler.inverse_transform(data_tr.drop('blueWins', axis = 1)),
                               columns = data_tr.drop('blueWins', axis = 1).columns)
data_not_scaled['blueWins'] = data_tr['blueWins']

In [ ]:
from sklearn.cluster import KMeans

timer = TimeIt()
k_means_list = [KMeans(n_clusters = k, random_state = 42).fit(data_copy.values) for k in range(2,6)]
timer.stop()

In [ ]:
# now we plot it
kmeans_cluster = Clusters('K-Means', k_means_list, data_not_scaled)
kmeans_cluster.plot()

[back to top](#top)

<a id = 'affinity'></a>
## Affinity Propagation

From wikipedia, "affinity propagation is a clustering algorithm based on the concept of 'message passing' between data points. Unlike clustering algorithms such as k-means or k-medoids, affinity propagation does not require the number of clusters to be determined or estimated before running the algorithm. Similar to k-mediods, affinity propagation finds 'exemplars', members of the input set that are representative of clusters."

https://en.wikipedia.org/wiki/Affinity_propagation


We will run 4 different Affinity Propagation algorithms with different values for the damping hyperparameter.

In [ ]:
from sklearn.cluster import AffinityPropagation

timer = TimeIt()
AP_list = [AffinityPropagation(damping = 0.5 + i*0.05, random_state = 42).fit(data_copy.values) for i in range(4)]
timer.stop()

In [ ]:
# now we plot
AP_clusters = Clusters('Affinity Propagation', AP_list, data_not_scaled)
AP_clusters.plot()

[back to top](#top)

<a id = 'mean_shift'></a>
## Mean Shift

According to the sklearn documentation, "MeanShift clustering aims to discover blobs in a smooth density of samples. It is a centroid based algorithm, which works by updating candidates for centroids to be the mean of the points within a given region."

The hyperparameter for mean-shift is the bandwidth. We will run 4 different Mean Shift algorithms with different values for the bandwidth.

In [ ]:
from sklearn.cluster import MeanShift

timer = TimeIt()
MS_list = [MeanShift(bandwidth = 1.25*i).fit(data_copy.values) for i in range(1,5)]
timer.stop()

In [ ]:
MS_clusters = Clusters('Mean Shift', MS_list, data_not_scaled)
MS_clusters.plot()

[back to top](#top)

<a id = 'spectral'></a>
## Spectral Clustering

According to the sklearn documentation: "Spectral Clustering performs a low-dimensional embedding of the affinity matrix between samples, followed by clustering, e.g., by KMeans, of the components of the eigenvectors in the low dimensional space".

The hypyerparameter for spectral clustering is number of clusters. We will run 4 different Spectral Clustering algorithms with 2,3,4 and 5 for the number of clusters.

In [ ]:
from sklearn.cluster import SpectralClustering

timer = TimeIt()
SC_list = [SpectralClustering(random_state = 42, n_clusters = i).fit(data_copy.values) for i in range(2,6)]
timer.stop()

In [ ]:
SC_clusters = Clusters('Spectral Clustering', SC_list, data_not_scaled)
SC_clusters.plot()

[back to top](#top)

<a id = 'agglomerative'></a>
## Agglomerative Clustering

According to the sklearn documentation: "Hierarchical clustering is a general family of clustering algorithms that build nested clusters by merging or splitting them successively. This hierarchy of clusters is represented as a tree (or dendrogram). The root of the tree is the unique cluster that gathers all the samples, the leaves being the clusters with only one sample."

We will be experimenting with number of clusters and the linkage parameters. The different linkage parameters are: ward, complete, average, and single.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

<a id = 'ward'></a>
### Ward Clustering

In [ ]:
timer = TimeIt()
Ward_list = [AgglomerativeClustering(n_clusters = i, linkage = 'ward').fit(data_copy.values) for i in range(2,6)]
timer.stop()

In [ ]:
Ward_clusters = Clusters('Ward Clustering', Ward_list, data_not_scaled)
Ward_clusters.plot()

[back to top](#top)

<a id = 'complete'></a>
### Complete Clustering

In [ ]:
timer = TimeIt()
Complete_list = [AgglomerativeClustering(n_clusters = i, linkage = 'complete').fit(data_copy.values) for i in range(2,6)]
timer.stop()

In [ ]:
Complete_clusters = Clusters('Complete Clustering', Complete_list, data_not_scaled)
Complete_clusters.plot()

[back to top](#top)

<a id = 'average'></a>
### Average Clustering

In [ ]:
timer = TimeIt()
Average_list = [AgglomerativeClustering(n_clusters = i, linkage = 'average').fit(data_copy.values) for i in range(2,6)]
timer.stop()

In [ ]:
Average_clusters = Clusters('Average Clustering', Average_list, data_not_scaled)
Average_clusters.plot()

[back to top](#top)

<a id = 'single'></a>
### Single Clustering

In [ ]:
timer = TimeIt()
Single_list = [AgglomerativeClustering(n_clusters = i, linkage = 'single').fit(data_copy.values) for i in range(2,6)]
timer.stop()

In [ ]:
Single_clusters = Clusters('Single Clustering', Single_list, data_not_scaled)
Single_clusters.plot()

[back to top](#top)

<a id = 'dbscan'></a>
## DBSCAN
According to the sklearn documentation: "The DBSCAN algorithm views clusters as areas of high density separated by areas of low density. Due to this rather generic view, clusters found by DBSCAN can be any shape, as opposed to k-means which assumes that clusters are convex shaped. The central component to the DBSCAN is the concept of core samples, which are samples that are in areas of high density. A cluster is therefore a set of core samples, each close to each other (measured by some distance measure) and a set of non-core samples that are close to a core sample (but are not themselves core samples). There are two parameters to the algorithm, min_samples and eps, which define formally what we mean when we say dense. Higher min_samples or lower eps indicate higher density necessary to form a cluster."

In [ ]:
from sklearn.cluster import DBSCAN

timer = TimeIt()
DB_list = [DBSCAN(eps = 2 + 0.25*i).fit(data_copy.values) for i in range(1,5)]
timer.stop()

In [ ]:
DB_clusters = Clusters('DBSCAN Clustering', DB_list, data_not_scaled)
DB_clusters.plot()

[back to top](#top)

<a id = 'optics'></a>
## OPTICS

According to the sklearn documentation: "The OPTICS algorithm shares many similarities with the DBSCAN algorithm, and can be considered a generalization of DBSCAN that relaxes the eps requirement from a single value to a value range. The key difference between DBSCAN and OPTICS is that the OPTICS algorithm builds a reachability graph, which assigns each sample both a reachability_ distance, and a spot within the cluster ordering_ attribute; these two attributes are assigned when the model is fitted, and are used to determine cluster membership. If OPTICS is run with the default value of inf set for max_eps, then DBSCAN style cluster extraction can be performed repeatedly in linear time for any given eps value using the cluster_optics_dbscan method. Setting max_eps to a lower value will result in shorter run times, and can be thought of as the maximum neighborhood radius from each point to find other potential reachable points."

In [ ]:
from sklearn.cluster import OPTICS

timer = TimeIt()
opt_list = [OPTICS(min_samples = i).fit(data_copy.values) for i in range(2,6)]
timer.stop()

In [ ]:
opt_clusters = Clusters('OPTICS Clustering', opt_list, data_not_scaled)
opt_clusters.plot()

[back to top](#top)

<a id = 'birch'></a>
## Birch

According to the sklearn documentation: "The Birch builds a tree called the Clustering Feature Tree (CFT) for the given data. The data is essentially lossy compressed to a set of Clustering Feature nodes (CF Nodes). The CF Nodes have a number of subclusters called Clustering Feature subclusters (CF Subclusters) and these CF Subclusters located in the non-terminal CF Nodes can have CF Nodes as children."

In [ ]:
from sklearn.cluster import Birch

timer = TimeIt()
birch_list = [Birch(n_clusters = i).fit(data_copy.values) for i in range(2,6)]
timer.stop()

In [ ]:
birch_clusters = Clusters('Birch Clustering', birch_list, data_not_scaled)
birch_clusters.plot()

[back to top](#top)

<a id = 'discussion'></a>
## Clustering Discussion

Of all of the clustering algorithms that we saw, the one that clustered the games best was the K-Means Clustering algorithm. This algorithm converged to a solution for all four cases in 6 seconds, which was much faster than the other clustering algorithms.

If we take a closer look at the documentation, we can see that SKlean mentions that the algorithm works best when we suspect that there are a small amount of clusters.

[back to top](#top)

<a id = 'advantage'></a>
## Team Advantage Analysis

Before we end this notebook, I wanted to spend time to create a column for which team has the advantage. I will define advantage as the team that has the higher combined stats after the first 10 minutes into the game. For example: we will say that blue team has the advantage in a game if they have more kills, less deaths, more gold, more experience, etc.

We begin by defining a column for the team's vision score. In the game, vision is controlled by placing vision wards that allow the team to see an area on the map that was previously hidden. The enemy team can destroy wards and deny visibility to some areas on the map. Having better vision of the map gives the team an advantage, so I think it is necessary to define this column.

In [ ]:
data_not_scaled['blueVisionScore'] = data_not_scaled['blueWardsPlaced'] + data_not_scaled['redWardsDestroyed']
data_not_scaled['redVisionScore'] = data_not_scaled['redWardsPlaced'] + data_not_scaled['blueWardsDestroyed']
# make sure tha columns were added
data_not_scaled.columns

To create the advantage column, we should compare (blue kills and red kills), 
(blue assists and red assists), (blue elite monsters and red elite monsters), 
(blue dragons and red dragons), (blue heralds and red heralds),
(blue towers destroyed and red towers destroyed),
(blue avg level and red avg level),
(blue total minions killed and red total minions killed), (blue jungle minions killed and red jungle minions killed),
(gold difference), (experience difference), (blue cs per min and red cs per min), (blue gold per min and red gold per min),
and (vision score)

In [ ]:
"""
I am hiding this cell and the following cell because they don't look clean.
If anyone has an idea of how to produce these columns in a better way,
please let me know!
"""
# first we define a column that contains points for how advantaged the team is
timer = TimeIt()
data_not_scaled['blueAdvantagePoints'] = [0]*data_not_scaled.shape[0]
for ind in range(data_not_scaled.shape[0]):
    data_not_scaled.loc[ind ,'blueAdvantagePoints'] = int(data_not_scaled['blueKills'].iloc[ind] > data_not_scaled['blueDeaths'].iloc[ind]) + int(data_not_scaled['blueAssists'].iloc[ind] > data_not_scaled['redAssists'].iloc[ind]) + int(data_not_scaled['blueEliteMonsters'].iloc[ind] > data_not_scaled['redEliteMonsters'].iloc[ind]) + int(data_not_scaled['blueDragons'].iloc[ind] > data_not_scaled['redDragons'].iloc[ind]) + int(data_not_scaled['blueHeralds'].iloc[ind] > data_not_scaled['redHeralds'].iloc[ind]) + int(data_not_scaled['redTowersDestroyed'].iloc[ind] > data_not_scaled['blueTowersDestroyed'].iloc[ind]) + int(data_not_scaled['blueAvgLevel'].iloc[ind] > data_not_scaled['redAvgLevel'].iloc[ind]) + int(data_not_scaled['blueTotalMinionsKilled'].iloc[ind] > data_not_scaled['redTotalMinionsKilled'].iloc[ind]) + int(data_not_scaled['blueTotalJungleMinionsKilled'].iloc[ind] > data_not_scaled['redTotalJungleMinionsKilled'].iloc[ind]) + int(data_not_scaled['blueGoldDiff'].iloc[ind] > 0) + int(data_not_scaled['blueExperienceDiff'].iloc[ind] > 0) + int(data_not_scaled['blueCSPerMin'].iloc[ind] > data_not_scaled['redCSPerMin'].iloc[ind]) + int(data_not_scaled['blueGoldPerMin'].iloc[ind] > data_not_scaled['redGoldPerMin'].iloc[ind]) + int(data_not_scaled['blueVisionScore'].iloc[ind] > data_not_scaled['redVisionScore'].iloc[ind])
timer.stop()

In [ ]:
# now the red team
timer = TimeIt()
data_not_scaled['redAdvantagePoints'] = [0]*data_not_scaled.shape[0]
for ind in range(data_not_scaled.shape[0]):
    data_not_scaled.loc[ind ,'redAdvantagePoints'] = int(data_not_scaled['blueKills'].iloc[ind] < data_not_scaled['blueDeaths'].iloc[ind]) + int(data_not_scaled['blueAssists'].iloc[ind] < data_not_scaled['redAssists'].iloc[ind]) + int(data_not_scaled['blueEliteMonsters'].iloc[ind] < data_not_scaled['redEliteMonsters'].iloc[ind]) + int(data_not_scaled['blueDragons'].iloc[ind] < data_not_scaled['redDragons'].iloc[ind]) + int(data_not_scaled['blueHeralds'].iloc[ind] < data_not_scaled['redHeralds'].iloc[ind]) + int(data_not_scaled['redTowersDestroyed'].iloc[ind] < data_not_scaled['blueTowersDestroyed'].iloc[ind]) + int(data_not_scaled['blueAvgLevel'].iloc[ind] < data_not_scaled['redAvgLevel'].iloc[ind]) + int(data_not_scaled['blueTotalMinionsKilled'].iloc[ind] < data_not_scaled['redTotalMinionsKilled'].iloc[ind]) + int(data_not_scaled['blueTotalJungleMinionsKilled'].iloc[ind] < data_not_scaled['redTotalJungleMinionsKilled'].iloc[ind]) + int(data_not_scaled['blueGoldDiff'].iloc[ind] < 0) + int(data_not_scaled['blueExperienceDiff'].iloc[ind] < 0) + int(data_not_scaled['blueCSPerMin'].iloc[ind] < data_not_scaled['redCSPerMin'].iloc[ind]) + int(data_not_scaled['blueGoldPerMin'].iloc[ind] < data_not_scaled['redGoldPerMin'].iloc[ind]) + int(data_not_scaled['blueVisionScore'].iloc[ind] < data_not_scaled['redVisionScore'].iloc[ind])
timer.stop()

In [ ]:
# let's look at the columns we just created
data_not_scaled.loc[:5, ('blueAdvantagePoints', 'redAdvantagePoints')]

In [ ]:
# finally, we make a column defining whether blue team has the advantage or not.
timer = TimeIt()
data_not_scaled['blueAdvantage'] = [np.nan]*data_not_scaled.shape[0]
for ind in range(data_not_scaled.shape[0]):
    if data_not_scaled.loc[ind, 'blueAdvantagePoints'] > data_not_scaled.loc[ind, 'redAdvantagePoints']:
        data_not_scaled.loc[ind, 'blueAdvantage'] = 1
    elif data_not_scaled.loc[ind, 'blueAdvantagePoints'] < data_not_scaled.loc[ind, 'redAdvantagePoints']:
        data_not_scaled.loc[ind, 'blueAdvantage'] = 0
    else:
        data_not_scaled.loc[ind, 'blueAdvantage'] = np.nan
timer.stop()

In [ ]:
data_not_scaled.loc[:5, ('blueAdvantagePoints', 'redAdvantagePoints', 'blueAdvantage')]

Now that we have the advantage column defined, we can see what percent of games blue team lost when they had the advantage.

In [ ]:
# I suspect it is around 20-25%
blue_loss_with_advantage = data_not_scaled[(data_not_scaled['blueWins'] == 0) & (data_not_scaled['blueAdvantage'] == 1)].shape[0]/data_not_scaled.shape[0]
red_loss_with_advantage = data_not_scaled[(data_not_scaled['blueWins'] == 1) & (data_not_scaled['blueAdvantage'] == 0)].shape[0]/data_not_scaled.shape[0]

print('Percent of games where blue lost with the advantage: {:.2f}%\n'.format(blue_loss_with_advantage*100))
print('Percent of games where red lost with the advantage: {:.2f}%'.format(red_loss_with_advantage*100))

[back to top](#top)